In [174]:
# pandas to handle dataframes
import pandas as pd
import json
# regular expressions
import re

In [175]:
# detect start of item [ID_number]
startItem = re.compile("^\[[0-9]+\]")
# name of item
startItemName = re.compile("^identifiedDisplayName")
startItemType = re.compile("Type:\s?")
itemAttack = re.compile("Attack:")
itemMagicAtk = re.compile("MagicAtk:")
itemReqLv = re.compile("Required Level:")
itemWeight = re.compile("Weight:")
# Daggers innate
daggerDblAtk = re.compile("Double Attack Lv:")
# Swords innate
swordDefPierce = re.compile("Defense Pierce:")
# Draco innate
spearDragonThrust = re.compile("Dragon Thrust:")
# Decks innate bonuses
deckSpCostReduc = re.compile("SP Cost Reduction:")
deckSpDisable = re.compile("No SP Regen while equipped")
# Shields innate
shieldAutoGuardLv = re.compile("Auto Guard Lv:")
# 2-h shields innate
shield2HMeleeReflect = re.compile("Reflect Melee Damage:")
# Axes innate
axeSplashRange = re.compile("Splash Range:")
# Katar innate
katarCritDmg = re.compile("Critical Damage Bonus:")
# Bows innate
bowAttackRange = re.compile("Attack Range:")
bowSpCost = re.compile("Attack SP Cost:")
# Scythes innate
scytheLeechRate = re.compile("Leech Rate:")
scytheLeechPower = re.compile("Leech Power:")
# Revolvers innate
revolverSpCost = re.compile("Attack SP Cost:")
revolverSpecial = re.compile("Special:")
# armor pieces (armors, shoes, garments, shields,..)
defenceArmor = re.compile("Defense:")
magicDefArmor = re.compile("MagicDef:")
# Shuffling deck innate
shufflingDeckInnate = re.compile("Disables Skill Use.")
shufflingDeckInnate2 = re.compile("Enables SP Recovery while equipped.")
# number of slot of items
slotCount = re.compile("slotCount")
# DK weapon innate
SwordDKInnate = re.compile("Vengeance Rate:")
# new draco weapon innate
DragonClawInnate = re.compile("Dragon Claw Rate:")

# Armors innate (HP/SP)
armorHPInnate = re.compile("HP Bonus:")
armorSPInnate = re.compile("SP Bonus:")
# Garment innate
garmentInnate = re.compile("Perfect Dodge:")
# Shoes innate
shoesInnate = re.compile("Move Speed:")
# Cards location (where it slots)
cardInnateLocation = re.compile("Location:")
# Item description
startItemDescription = re.compile("^identifiedDescriptionName")
startDescription = False
startDescriptionText = False

itemsWithInnate = []
newItem = {}
with open("itemInfo_EN.lua", "r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        line = re.sub(r"\"","",line)
        line = re.sub(r"\^[a-zA-Z0-9]{6}","",line)
        line = re.sub(r"\\t","",line)
        line = re.sub(r",","",line)
        line = line.strip()
        
        # Start of new item, we create a new empty dict
        if startItem.match(line):
            if(newItem and ((newItem['innate']) or (newItem['type'] and 'Headgear' in newItem['type']) or (newItem['type'] and 'Accessory' in newItem['type']))):
                itemsWithInnate.append(newItem)
            #print(newItem)
            itemId = line.split("[")[1].split("]")[0]
            newItem ={
                "ID" : int(itemId),
                "name" : None,
                "slots" : None,
                "type" : None,
                "weapLv" : None,
                "attack": None,
                "magicAtk" : None,
                "reqLv" : None,
                "weight" : None,
                "innate" : None,
                "defense" : None,
                "magicDef" : None,
                "desc" : ""    
            }
            startDescription = False
            startDescriptionText = False
        # Item name
        if startItemName.match(line):
            if len(line.split(" = "))>1:
                newItem['name'] = line.split(" = ")[1]
        # Item type
        if startItemType.match(line):
            newItem["type"] = re.sub(startItemType,"", line)
            if "Lv:" in newItem["type"]:     
                firstPart = newItem["type"].split("Lv")[0].strip()
                newItem["weapLv"] = int(newItem["type"][-1])
                newItem["type"] = firstPart
        # Item attack (weapons) - sometimes MagicAtk appear on the same line...
        if itemAttack.match(line):
            newItem["attack"] = line.replace("Attack: ","").strip()
            if "MagicAtk" in newItem["attack"]: #sometimes MagicAtk appear on the same line...
                newItem["magicAtk"] = newItem["attack"][-3:]
                newItem["attack"] = newItem["attack"].split("MagicAtk")[0]
        # Magic attack
        if itemMagicAtk.match(line):
            newItem["magicAtk"] = line.split("MagicAtk:")[1].strip()
        # Required level of item
        if itemReqLv.match(line):
            newItem["reqLv"] = line.split("Required Level:")[1].strip()
            if newItem['reqLv']=='None':
                newItem['reqLv']=0
        # item weight, marks only the end of item infos since its last line
        if itemWeight.match(line):
            newItem["weight"] = line.split("Weight:")[1].strip()
            startDescription = False
            startDescriptionText = False
        # dagger innate
        if daggerDblAtk.match(line):
            newItem["innate"] = line.replace(": ",". ")
        # sword innate
        if swordDefPierce.match(line):
            newItem["innate"] = line.replace(":","")
        # spear innate
        if spearDragonThrust.match(line):
            newItem["innate"] = line.replace(":","")
        # deck innates
        if deckSpCostReduc.match(line):
            newItem["innate"] = line.replace(":","")
        if deckSpDisable.match(line):
            newItem["innate"] = f"{newItem['innate']}\n{line}"
        # shields innate both 1handed and 2handed
        if shieldAutoGuardLv.match(line):
            newItem["innate"] = line.replace(":",".")
        if shield2HMeleeReflect.match(line):
            newItem["innate"] = line.replace(":",".")
        # axe innate
        if axeSplashRange.match(line):
            newItem["innate"] = line
        # katar innate
        if katarCritDmg.match(line):
            newItem["innate"] = line
        # bows innate
        if bowAttackRange.match(line) and newItem["type"] and ("Bow" in newItem["type"]):
            newItem["innate"] = line.replace(":","")
        if bowSpCost.match(line) and newItem["type"] and ("Bow" in newItem["type"]):
            newItem["innate"] = f"{newItem['innate']}\n{line}"
        # scythe innate
        if scytheLeechRate.match(line):
            newItem["innate"] = line.replace(":","")
        if scytheLeechPower.match(line):
            newItem["innate"] = f"{newItem['innate']}\n{line.replace(':','')}"
        # revolver innate
        if revolverSpCost.match(line) and newItem["type"]=="Revolver":
            newItem["innate"] = line
        if revolverSpecial.match(line) and newItem["type"]=="Revolver":
            newItem["innate"] = f"{newItem['innate']}\n{line}"
        # dk swords
        if SwordDKInnate.match(line):
            newItem["innate"] = line.replace(":",".")
        # draco spears
        if DragonClawInnate.match(line):
            newItem["innate"] = line.replace(":",".")
        # DEF and MDEF of armors (armor,garment, shoes,...)
        if defenceArmor.match(line):
            newItem["defense"] = line.split("Defense:")[1].strip()
        if magicDefArmor.match(line):
            newItem["magicDef"] = line.split("MagicDef:")[1].strip()
        # Shuffling deck innate
        if shufflingDeckInnate.match(line):
            newItem["innate"] = line
        if shufflingDeckInnate2.match(line):
            newItem["innate"] = f"{newItem['innate']}\n{line}"
        # Armors innates
        if armorHPInnate.match(line):
            newItem["innate"] = line
        if armorSPInnate.match(line):
            newItem["innate"] = f"{newItem['innate']}\n{line}"
        # Garment innates
        if garmentInnate.match(line):
            newItem["innate"] = line
        # Shoes innates
        if shoesInnate.match(line):
            newItem["innate"] = line
        # Cards location
        if cardInnateLocation.match(line):
            newItem["innate"] = line.split("Location:")[1].strip()
        # Slots count
        if slotCount.match(line):
            newItem["slots"] = line.split(" = ")[1].strip()
        # Item description
        if startItemDescription.match(line):
            startDescription = True
            continue
        # the "______" generally mark the start of description
        if "_______" in line and (startDescription) and not(startDescriptionText):
            startDescriptionText = True
            continue
        # if we are reading the description and we find another "_____" it marks the end of description but only for cards
        if ("_______" in line) and (startDescription) and (startDescriptionText):
            if newItem['type']=="Card":
                startDescription = False
                startDescriptionText = False
                continue
        # detect start of description
        if startDescription and startDescriptionText:
            if not "____" in line:
                if newItem['desc']=="":
                    newItem['desc'] = line
                else:
                    newItem["desc"] = f"{newItem['desc']}\n{line}"
f.close()

## Add item locations by merging infos with "equiLocations" excel sheet

In [183]:
equips = pd.DataFrame.from_dict(itemsWithInnate)

In [184]:
# bring equip locations from a different file!! update it to generate new items
# because itemlua file contains all RO items ( even the ones not implemented!)
equipLocations = pd.read_excel(open('equipLocation_updated_full2.xlsx', 'rb'), sheet_name='UniqueWeaps')[['ID','ObtainedFrom']]

In [185]:
equips = pd.merge(equips, equipLocations, on="ID", how="left")

In [186]:
equips['type'].unique()

array(['Heavy Shoes', 'Sword', 'Bone Sword', 'Fire Sword', 'Long Sword',
       'Water Sword', 'Knight Sword', 'Ghost Sword',
       'Poison Knight Sword', 'Wind Long Sword', 'Water Long Sword',
       'Holy Sword', 'Dagger', 'Dark Dagger', 'Poison Dagger',
       'Holy Dagger', 'Fire Dagger', 'Earth Dagger', 'Water Dagger',
       'Katar', 'Water Katar', 'Earth Katar', 'Fire Katar', 'Wind Katar',
       'Dark Katar', 'Axe', 'Poison Axe', 'Scythe', 'Bow',
       'Medium Shield', 'Upper Headgear', 'Spear', 'Wyrm Spear',
       'Water Wyrm Spear', 'Poison Wyrm Spear', 'Wind Wyrm Spear',
       'Dark Scythe', 'Holy Scythe', 'Deck', 'Shuffling Deck',
       'Heavy Bow', 'Heavy Shield', 'Round Shield', 'Arm Shield',
       'Square Shield', 'Colossal Shield', 'Prowler Gear',
       'Offhand Dagger', 'Mid Headgear', 'Headgear', ' Lower Headgear',
       'Mid/Low Headgear', 'Simple Armor', 'Medium Armor', 'Light Armor',
       'Heavy Armor', 'Medium Garment', 'Unique Armor', 'Light Garment',
 

# Weapons

In [187]:
swords = equips[equips['type'].isin(["Bone Sword","Sword","Fire Sword","Long Sword","Water Sword","Ghost Sword","Holy Sword","Knight Sword","Poison Knight Sword", "Wind Long Sword","Water Long Sword","Dark Knight Sword"])]
daggers = equips[equips['type'].isin(["Dagger","Dark Dagger","Poison Dagger","Fire Dagger",
                                        "Earth Dagger","Water Dagger","Wind Dagger","Holy Dagger"])]
bows = equips[equips['type'].isin(["Bow","Heavy Bow"])]
axes = equips[equips['type'].isin(["Axe","Poison Axe"])]
scythes = equips[equips['type'].isin(["Scythe","Dark Scythe","Fire Scythe","Ghost Scythe","Holy Scythe"])]
katars = equips[equips['type'].isin(["Katar","Dark Katar","Fire Katar",
                                        "Earth Katar","Water Katar","Wind Katar"])]
spears = equips[equips['type'].isin(["Spear","Wyrm Spear","Poison Wyrm Spear","Wind Wyrm Spear","Water Wyrm Spear"])]
decks = equips[equips['type'].isin(["Deck","Shuffling Deck"])]
revolvers = equips[equips['type'].isin(["Revolver"])]


### change first line with the appropriate weapon u want to generate wiki code for

In [147]:
weapons = swords
weapons = weapons.astype({'reqLv': 'int32',
                        'weapLv':'int32',
                        'attack':'int32',
                        'magicAtk':'int32'})
weapons = weapons.sort_values(by=['reqLv','attack'])
lineBreak = "\n"
possibleElements = ['Fire','Water','Ghost','Holy','Dark','Poison','Earth','Wind']
import math
for i in range(len(weapons)):
    weapon = weapons.iloc[i]
    if pd.isnull(weapon['ObtainedFrom']):
        continue
    print("|-")
    print(f"||'''{weapon['name']} [{weapon['slots']}]'''")
    print(f"||{weapon['reqLv']}")
    print(f"||{weapon['attack']}")
    print(f"||{weapon['magicAtk']}")
    print(f"||{int(weapon['weapLv'])}")
    # Add element in description
    if any(elem in weapon['type'] for elem in possibleElements):
        weapon["desc"] = f"{weapon['type']}\n{weapon['desc']}"

    weapon['desc'] = weapon['desc'].replace(lineBreak,'<br>')
    weapon['desc'] = weapon['desc'].replace('Set Bonus:',"'''[Set Bonus]'''")
    print(f"||{weapon['desc']}")
    print(f"||{weapon['innate'].replace(lineBreak,'<br>')}")
    if (i+1)<len(weapons) and (weapon['name']==weapons.iloc[i+1]['name']):
        print(f"||[[NPC_Equipment|NPC Shop]]")
    elif pd.isnull(weapon['ObtainedFrom']):
        print("||")
    else:
        print(f"||{weapon['ObtainedFrom']}")
    

|-
||'''Daehyon Sword [1]'''
||0
||45
||0
||4
||HP +10%<br>Dragon Breath DMG+5% per refine<br>ASPD-20
||Dragon Claw Rate. 20%
||Sunken Temple
|-
||'''Sword of the Orphan [0]'''
||1
||7
||7
||1
||Cannot be Refined<br>EXP Received +10%<br>SP Regen+20%
||Defense Pierce 10%
||Orphan Voucher
|-
||'''Curse of the Orphan [0]'''
||1
||9
||0
||1
||Dark Knight Sword<br>Cannot be Refined<br>Dark Element<br>EXP Received +10%<br>HP Regen+40%<br>Cannot be Worn past Lv30
||Vengeance Rate. 10%
||Orphan Voucher
|-
||'''Lost Sword [2]'''
||15
||15
||6
||4
||HP+10%<br>Rolling Cutter DMG +15%<br>Rolling Cutter DMG+1% per Refine<br>+Extra 1 ATK per Refine
||Defense Pierce 10%
||Pyramid Basement - MVP
|-
||'''Rotten Cutter [2]'''
||15
||23
||0
||3
||Envenom DMG+25%<br>SP Consumption -5%
||Defense Pierce 15%
||Geffen Dungeon
|-
||'''Sailor Sword [2]'''
||25
||15
||0
||2
||Rolling Cutter DMG+20%<br>Pirate Swords Set:<br>Sailor and Marauder<br>'''[Set Bonus]'''<br>ATK +10
||Defense Pierce 10%
||Sunken Ship
|-


C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\3049607073.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weapon['desc'] = weapon['desc'].replace(lineBreak,'<br>')
C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\3049607073.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weapon['desc'] = weapon['desc'].replace('Set Bonus:',"'''[Set Bonus]'''")
C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\3049607073.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

# Lefthand items (shields, armguards..)

In [150]:
shields = equips[equips['type'].isin(["Medium Shield","Heavy Shield","Round Shield","Arm Shield","Square Shield","Shield","Colossal Shield"])]
prowler_gear = equips[equips['type'].isin(["Prowler Gear"])]
shadowseer_daggers = equips[equips['type'].isin(["Offhand Dagger"])]

### change first line with the appropriate weapon u want to generate wiki code for

In [152]:
lefthands = shields # <== change here to choose "shields", "prowler_gear" or "shadowseer_daggers"
lefthands = lefthands.astype({'reqLv': 'int32'})
lefthands = lefthands.sort_values(by=['reqLv'])
lineBreak = "\n"
possibleElements = ['Fire','Water','Ghost','Holy','Dark','Poison','Earth','Wind']
import math
for i in range(len(lefthands)):
    lefthand = lefthands.iloc[i]
    if pd.isnull(lefthand['ObtainedFrom']):
        continue
    #if (i+1)<len(equips) and (sword['name']==equips.iloc[i+1]['name']):
    #    continue
    print("|-")
    print(f"||'''{lefthand['name']} [{lefthand['slots']}]'''")
    print(f"||{lefthand['reqLv']}")
    print(f"||{lefthand['defense']}")
    print(f"||{lefthand['magicDef']}")
    print(f"||{lefthand['weight']}")
    
    lefthand['desc'] = lefthand['desc'].replace(lineBreak,'<br>')
    lefthand['desc'] = lefthand['desc'].replace('Set Bonus:',"'''[Set Bonus]'''")
    print(f"||{lefthand['desc']}")
    print(f"||{lefthand['innate'].replace(lineBreak,'<br>')}")
    if (i+1)<len(lefthands) and (lefthand['name']==lefthands.iloc[i+1]['name']):
        print(f"||[[NPC_Equipment|NPC Shop]]")
    elif pd.isnull(lefthand['ObtainedFrom']):
        print("||")
    else:
        print(f"||{lefthand['ObtainedFrom']}")

|-
||'''Shield of the Orphan [0]'''
||1
||010
||003
||50
||Cannot be Refined<br>HP+50<br>EXP Received +10%<br>SP Regen+20%<br>Cannot be Worn past Lv30
||Auto Guard Lv. 3
||Orphan Voucher
|-
||'''Golden Buckler [1]'''
||10
||010
||010
||80
||HP Regen +60%<br>VIT+2
||Auto Guard Lv. 6
||Prontera Culvert
|-
||'''Lost Bunny [1]'''
||15
||020
||007
||50
||All Stats +2<br>Every 2 refines:<br>Max HP +1%<br>Max SP +1%
||Auto Guard Lv. 5
||Pyramid Basement - MVP
|-
||'''Mad Bunny [0]'''
||15
||015
||005
||50
||All Stats +1
||Auto Guard Lv. 4
||Pyramid Basement - MVP
|-
||'''Iron Shield [0]'''
||25
||030
||000
||150
||Shield Boomerand DMG+25%<br>DEF+4%
||Auto Guard Lv. 8
||Byalan Island
|-
||'''Silver Guard [1]'''
||40
||010
||002
||80
||Flat Defense +10%<br>HP+100<br>SP+10
||Auto Guard Lv. 10
||Kobold Leader
|-
||'''Clock Shield [1]'''
||40
||015
||005
||80
||Move Speed +10%<br>Flee +15
||Auto Guard Lv. 4
||Clock Tower
|-
||'''Giant Shield [1]'''
||45
||010
||001
||200
||Immune to Knockback.<br>

C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\1383762501.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lefthand['desc'] = lefthand['desc'].replace(lineBreak,'<br>')
C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\1383762501.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lefthand['desc'] = lefthand['desc'].replace('Set Bonus:',"'''[Set Bonus]'''")


# Armors

In [156]:
armors = equips[equips['type'].str.contains("Armor")]
garments = equips[equips['type'].str.contains("Garment")]
shoes = equips[equips['type'].str.contains("Shoes")]

### change first line with the appropriate armor u want to generate wiki code for

In [159]:
armors_pieces = shoes ### change first line with the appropriate weapon u want to generate wiki code for
armors_pieces = armors_pieces.astype({'reqLv': 'int32',
                        'defense':'int32',
                        'magicDef':'int32'
                       })
armors_pieces = armors_pieces.sort_values(by=['reqLv'])
lineBreak = "\n"
possibleElements = ['Fire','Water','Ghost','Holy','Dark','Poison','Earth','Wind']
import math
for i in range(len(armors_pieces)):
    armor = armors_pieces.iloc[i]
    if pd.isnull(armor['ObtainedFrom']):
        continue
    print("|-")
    print(f"||'''{armor['name']} [{armor['slots']}]'''")
    print(f"||{armor['reqLv']}")
    print(f"||{armor['defense']}")
    print(f"||{armor['magicDef']}")


    armor['desc'] = armor['desc'].replace(lineBreak,'<br>')
    armor['desc'] = armor['desc'].replace('Set Bonus:',"'''[Set Bonus]'''")
    print(f"||{armor['desc']}")
    print(f"||{armor['innate'].replace(lineBreak,'<br>')}")
    print(f"||[[{armor['ObtainedFrom']}]]")

|-
||'''Orphan Shoes [1]'''
||1
||4
||2
||SP+20<br>Young Orphan Set:<br>Clothes Cape and Shoes<br>'''[Set Bonus]'''<br>All Stats +1<br>Enables Heal Lv4<br>Exp Received +10%
||Move Speed: 003%
||[[Pyramid Basement]]
|-
||'''Tomb Thief Boots [1]'''
||10
||4
||2
||Backstab DMG+10%<br>Tomb Thief Set:<br>Armor Garment Shoes<br>'''[Set Bonus]'''<br>ATK+3<br>MATK+3<br>Enables Hiding and Increase Agility Lv3
||Move Speed: 005%
||[[Upper Pyramid]]
|-
||'''Scarlet Shoes [1]'''
||10
||6
||1
||ASPD+5%<br>Cast Time-5%<br>Scarlet Set:<br>Armor Garment Shoes<br>'''[Set Bonus]'''<br>EXP+5%<br>All Stats +1
||Move Speed: 006%
||[[Payon Cave]]
|-
||'''Running Shoes [0]'''
||10
||5
||1
||After Cast Delay -10%
||Move Speed: 015%
||[[Prontera Culvert]]
|-
||'''Arch Black Shoes [0]'''
||15
||3
||3
||Cast Time -15%
||Move Speed: 001%
||[[Society Nest Drop]]
|-
||'''Fluffy Fish Shoes [1]'''
||25
||5
||7
||HP+1%<br>1% per refine chance to freeze the attacker when hit.
||Move Speed: 010%
||[[Beach Dungeon]]
|-
|

C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\3953651282.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  armor['desc'] = armor['desc'].replace(lineBreak,'<br>')
C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\3953651282.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  armor['desc'] = armor['desc'].replace('Set Bonus:',"'''[Set Bonus]'''")


# Headgears

In [161]:
upper_headgears = equips[equips['type'].isin(["Headgear","Upper Headgear"])]
mid_headgears = equips[equips['type'].isin(["Mid Headgear"," Mid Headgear","id Headgear"])]
lower_headgear = equips[equips['type'].isin([" Lower Headgear","Lower Headgear","Low Headgear","Unique Lower Headgear","Unchained Lower Headgear","Lower Head"])]
uppermid_headgears = equips[equips['type'].isin(["Upper and Mid Headgear","Upper and Middle Headgear","Upper/Mid Headgear","Upper & Mid Headgear"])]
midlow_headgears = equips[equips['type'].isin(["Mid/Low Headgear","Unique Mid/Low Headgear"])]
uppermidlow_headgears = equips[equips['type'].isin(["Upper Mid Low Headgear","UpperMidLower Headgear","Upper/Mid/Low Headgear"])]

### change first line with the appropriate headgear u want to generate wiki code for

In [162]:
headgears = upper_headgears # <== change here to choose what to generate### change first line with the appropriate weapon u want to generate wiki code for
headgears = headgears[headgears.ObtainedFrom.notna()]
headgears = headgears.sort_values(by=['reqLv'])
lineBreak = "\n"
possibleElements = ['Fire','Water','Ghost','Holy','Dark','Poison','Earth','Wind']
import math
for i in range(len(headgears)):
    headgear = headgears.iloc[i]
    if pd.isnull(headgear['ObtainedFrom']):
        continue
        
    print("|-")
    print(f"||'''{headgear['name']} [{headgear['slots']}]'''")
    print(f"||{headgear['reqLv']}")
    print(f"||{headgear['type']}")
    print(f"||{headgear['defense']}")
    print(f"||{headgear['magicDef']}")


    headgear['desc'] = headgear['desc'].replace(lineBreak,'<br>')
    headgear['desc'] = headgear['desc'].replace('Set Bonus:',"'''[Set Bonus]'''")
    print(f"||{headgear['desc']}")
    print(f"||[[{headgear['ObtainedFrom']}]]")

|-
||'''Old Bandana [1]'''
||10
||Headgear
||004
||000
||Recover 50 HP per kill<br>Buried Set:<br>Accessory and Headgear<br>'''[Set Bonus]'''<br>All Stats +1
||[[Upper Pyramid]]
|-
||'''Pirate Bandana [0]'''
||10
||Headgear
||005
||000
||STR+3<br>Defense Pierce +3%<br>Pirate Set:<br>Headgear and Pirate Dagger<br>'''[Set Bonus]'''<br>ASPD +2
||[[Sunken Ship]]
|-
||'''Corsair [1]'''
||10
||Headgear
||010
||000
||STR+1 per refine<br>ATK+3<br>Pirate Set:<br>Headgear and Pirate Dagger<br>'''[Set Bonus]'''<br>ASPD +2
||[[Sunken Ship]]
|-
||'''Crown of the Divine [0]'''
||125
||Headgear
||005
||005
||Regen 1 SP per refine every second<br>After Cast Delay -10%
||[[Divine Temple of Freya]]
|-
||'''Mistress Crown [1]'''
||20
||Headgear
||003
||005
||SP Cost -10%<br>Mistress Set:<br>Hat and Card<br>'''[Set Bonus]'''<br>SP Cost-10%
||[[Mistress]]
|-
||'''Lady Tanee Doll [1]'''
||20
||Headgear
||005
||000
||Item Healing Power +15%<br>Lady Tanee Set:<br>Hat and Card<br>'''[Set Bonus]'''<br>Item Heal

C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\4291139217.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  headgear['desc'] = headgear['desc'].replace(lineBreak,'<br>')
C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\4291139217.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  headgear['desc'] = headgear['desc'].replace('Set Bonus:',"'''[Set Bonus]'''")


# Accessories

In [164]:
accessories = equips[equips['type'].isin(["Accessory"," Left Accessory","Left Accessory"," Right Accessory","Right Accessory","Unchained Right Accessory","Unique Accessory"])]

In [166]:
accessories = accessories[equips.ObtainedFrom.notna()]
accessories = accessories.astype({'reqLv': 'int32'})
accessories = accessories.sort_values(by=['reqLv'])
lineBreak = "\n"
import math
for i in range(len(accessories)):
    accessory = accessories.iloc[i]
    if pd.isnull(accessory['ObtainedFrom']):
        continue
        
    print("|-")
    print(f"||'''{accessory['name']} [{accessory['slots']}]'''")
    print(f"||{accessory['reqLv']}")

    accessory['desc'] = accessory['desc'].replace(lineBreak,'<br>')
    accessory['desc'] = accessory['desc'].replace('Set Bonus:',"'''[Set Bonus]'''")
    print(f"||{accessory['desc']}")
    print(f"||{accessory['type']}")
    print(f"||[[{accessory['ObtainedFrom']}]]")

|-
||'''Flower Ring [1]'''
||0
||Random stat bonus on drop
||Accessory
||[[Pyramid Basement]]
|-
||'''Ring of the Orphan [0]'''
||1
||EXP Received +10%<br>SP+10<br>ASPD+3<br>Cannot be Worn past Lv30
||Right Accessory
||[[Orphan Voucher]]
|-
||'''Old Tomb Ring [1]'''
||10
||Recovers 10 SP per kill<br>Buried Set:<br>Accessory and Headgear<br>'''[Set Bonus]'''<br>All Stats +1
||Accessory
||[[Upper Pyramid]]
|-
||'''Old Anklet [1]'''
||15
||Enables Potion Pitcher Lv2
||Accessory
||[[Society Nest Drop]]
|-
||'''Arch Ring [1]'''
||15
||Melee Attack+7%
||Accessory
||[[Society Nest Exchange]]
|-
||'''Arch Gloves [1]'''
||15
||Ranged Attack +7%
||Accessory
||[[Society Nest Exchange]]
|-
||'''Arch Brooch [1]'''
||15
||Magic Damage +7%
||Accessory
||[[Society Nest Exchange]]
|-
||'''Arch Necklace [1]'''
||15
||HP +7%
||Accessory
||[[Society Nest Exchange]]
|-
||'''Arch Earring [1]'''
||15
||SP +7%
||Accessory
||[[Society Nest Exchange]]
|-
||'''Arch Anklet [1]'''
||15
||MATK+1%<br>Enables Potion 

C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\1538543866.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  accessories = accessories[equips.ObtainedFrom.notna()]
C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\1538543866.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accessory['desc'] = accessory['desc'].replace(lineBreak,'<br>')
C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\1538543866.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accessory['desc'] = accessory['desc'].replace('Set Bonus:',"'''[Set Bonus]'''")


# Cards
### cards location are merged with the existing client infos coming from "all_cards_locations.xlsx" file

In [188]:
cards = equips[equips['type'].str.contains("Card")]
cardsLocations = pd.read_excel(open('all_cards_locations.xlsx', 'rb'))
cardsLocations['name'] = cardsLocations.apply(lambda row : f"{row['Name']} Card", axis=1)
cards = pd.merge(cards, cardsLocations, on="name", how="left")
cards = cards[['name',"innate","desc","Lv","Location","Location.1"]]

In [189]:
cards

,name,innate,desc,Lv,Location,Location.1
0,Poring Card,Armor,LUK+1,1.0,Normal,Poring Family
1,Fabre Card,Headgear,SP +5%,1.0,Normal,Prontera Fields
2,Pupa Card,Headgear,HP +5%,2.0,Normal,Prontera Fields
3,Drops Card,Weapon,DEX+1,5.0,Normal,Poring Family
4,Lunatic Card,Accessory,Critical Rate +2\nPerfect Dodge +1,3.0,Normal,Prontera Fields
5,Picky Card,Shoes,Flat DEF +5,1.0,Normal,Sograt Desert
6,Chonchon Card,Garment,Total Flee +3%,NaN,NaN,NaN
7,Willow Card,Weapon,SP+30,2.0,Normal,Payon Forest
8,Picky Egg Card,Armor,DEF +2%,NaN,NaN,NaN
9,Roda Frog Card,Headgear,HP +3%\nSP +3%,6.0,Normal,Prontera Fields


In [190]:
cards['innate'].unique()

array(['Armor', 'Headgear', 'Weapon', 'Accessory', 'Shoes', 'Garment',
       'Shield', 'Middle Headgear', 'Mid Headgear', 'Upper Headgear',
       'Shoes and Garment', 'Any Slot', 'Garment and Shoes',
       'Lower Headgear', 'Low Headgear', 'Acessory',
       'Any Refineable Slot', 'Any Shoes', 'Left Accessory',
       'Right Accessory', 'Lower Head'], dtype=object)

### change first line with the appropriate card slot u want to generate wiki code for

In [193]:
typesToShow= ["Accessory","Acessory","Any Slot"] # <= change this to choose slot to generate
cards = cards.sort_values(by=['name'])
lineBreak = "\n"
import math
for i in range(len(cards)):
    card = cards.iloc[i]
    if any(slot in card['innate'] for slot in typesToShow):
        print("|-")
        print(f"||[[File:CardIcon.gif|link=]]'''{card['name']}'''")
        card['desc'] = card['desc'].replace(lineBreak,'<br>')
        card['desc'] = card['desc'].replace('Set Bonus:',"'''[Set Bonus]'''") # add set bonus in bold
        print(f"||{card['desc']}")
        print(f"||{card['innate']}")
        if not math.isnan(card['Lv']): # mob level!
            print(f"||{int(card['Lv'])}")
        else:
            print(f"||")
        if(card['Location']=='MVP'): # add mvp gold icon if it's an mvp card
            print(f"||[[File:MvpIcon.png|link=]] {card['Location']}")
        elif(card['Location']=="Mini Boss"): # add mini boss blue icon if it's a mini boss card
            print(f"||[[File:BossIcon.png|link=]] {card['Location']}")
        else:
            print(f"||{card['Location']}")
        print(f"||{card['Location.1']}")


|-
||[[File:CardIcon.gif|link=]]'''Abyss Raydric Archer Card'''
||SP+5%
||Accessory
||
||nan
||nan
|-
||[[File:CardIcon.gif|link=]]'''Abyss Raydric Card'''
||HP+5%
||Accessory
||117
||Normal
||Abyss Glast Heim (Distortion)
|-
||[[File:CardIcon.gif|link=]]'''Agav Card'''
||Variable Cast Time -10%
||Accessory
||105
||Normal
||Rachel Sanctuary
|-
||[[File:CardIcon.gif|link=]]'''Ancient Tree Card'''
||ATK+1%
||Accessory
||99
||Champion
||Geffenia
|-
||[[File:CardIcon.gif|link=]]'''Apocalipse Card'''
||DMG vs Undead Element +5%
||Any Slot
||
||nan
||nan
|-
||[[File:CardIcon.gif|link=]]'''Archdam Card'''
||DMG vs Water Element +5%
||Any Slot
||92
||Normal
||Juperos
|-
||[[File:CardIcon.gif|link=]]'''Argiope Card'''
||Poison Magic DMG +5%
||Accessory
||51
||Normal
||Mjolnir Fields
|-
||[[File:CardIcon.gif|link=]]'''Ashring Card'''
||EXP+5% Doubled bonus if worn with Old Blue Bag Card.(Non-Stacking Bonus)
||Accessory
||100
||Normal
||Kafra Warehouse
|-
||[[File:CardIcon.gif|link=]]'''Aunoe Car

C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\3204423779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  card['desc'] = card['desc'].replace(lineBreak,'<br>')
C:\Users\Aniel\AppData\Local\Temp\ipykernel_16108\3204423779.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  card['desc'] = card['desc'].replace('Set Bonus:',"'''[Set Bonus]'''") # add set bonus in bold
